In [ ]:
# 2602112215 - Darrius Salim

In [ ]:
# !pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=da744bbb8b6a472f5fcaf409e37f5361ddbacf4dedfaa03fec4da5be8c505db8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import heapq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from rouge_score import rouge_scorer

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("data_3B.csv")

In [ ]:
# Ringkasan berbasis frekuensi kata
def generate_summary(text, top_n=3):
    stop_words = set(stopwords.words("indonesian"))
    words = word_tokenize(text.lower())
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    freq_table = {}
    for word in words:
        freq_table[word] = freq_table.get(word, 0) + 1

    sentences = sent_tokenize(text)
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in freq_table:
                sentence_scores[sentence] = sentence_scores.get(sentence, 0) + freq_table[word]

    best_sentences = heapq.nlargest(top_n, sentence_scores, key=sentence_scores.get)
    return ' '.join(best_sentences)

Summarization with Frequency Method:
- Preprocessing: Removes stopwords and punctuation.
- Frequency Table: Counts how often each word appears.
- Sentence Scoring: Sums word frequencies per sentence.
- Selection: Chooses top N (3 or 5) highest-scoring sentences.

In [ ]:
# Generate ringkasan
df['summary_top3'] = df['content'].apply(lambda x: generate_summary(x, top_n=3))
df['summary_top5'] = df['content'].apply(lambda x: generate_summary(x, top_n=5))

Generate Summaries
Applies the above technique to each content piece—returns:
- summary_top3: using top 3 sentences
- summary_top5: using top 5 sentences

In [ ]:
# Tampilkan hasil
df[['content', 'summary', 'summary_top3', 'summary_top5']]

,content,summary,summary_top3,summary_top5
0,"TEMPO.CO, Jakarta - Wakil Presiden Ma'ruf Amin...",Wakil Presiden Ma'ruf Amin meninjau lokasi keb...,"""Penyebab kejadian masih dalam proses investig...","""Penyebab kejadian masih dalam proses investig..."
1,INFO NASIONAL Memperingati 14 tahun PT Sarana...,Menteri Keuangan Sri Mulyani Indrawati dan jaj...,Terdapat tiga program utama yang akan menjadi ...,Terdapat tiga program utama yang akan menjadi ...
2,"TEMPO.CO, Jakarta -Wacana Presiden Joko Widodo...",Wacana Presiden Joko Widodo atau Jokowi 3 peri...,"Sebab itu, lembaga yang berwenang terhadap pem...","Sebab itu, lembaga yang berwenang terhadap pem..."


In [ ]:
# Evaluasi ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def compute_rouge_scores(preds, refs):
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for pred, ref in zip(preds, refs):
        result = scorer.score(ref, pred)
        for key in scores:
            scores[key].append(result[key].fmeasure)
    return {key: np.mean(scores[key]) for key in scores}

rouge_top3 = compute_rouge_scores(df['summary_top3'], df['summary'])
rouge_top5 = compute_rouge_scores(df['summary_top5'], df['summary'])

print("ROUGE Score - Top 3 Kalimat:", rouge_top3)
print("ROUGE Score - Top 5 Kalimat:", rouge_top5)

ROUGE Score - Top 3 Kalimat: {'rouge1': np.float64(0.2547083316245553), 'rouge2': np.float64(0.12059617634939057), 'rougeL': np.float64(0.1560601046762066)}
ROUGE Score - Top 5 Kalimat: {'rouge1': np.float64(0.24060012290028035), 'rouge2': np.float64(0.15635096293510933), 'rougeL': np.float64(0.17295690728849555)}


Interpretasi:
- Top 3 lebih padat dan ringkas, tetapi bisa kehilangan beberapa konteks penting—itulah mengapa ROUGE-1 lebih tinggi, namun ROUGE-2 dan L lebih rendah.
- Top 5 memberikan cakupan konten yang lebih luas dan koherensi yang lebih baik—cocok untuk ringkasan yang lebih informatif dan menyeluruh.

Analogi sederhananya: Top 3 seperti headline berita, sementara Top 5 seperti paragraf pembuka yang menjelaskan pokok bahasan dengan lebih lengkap.

LINK VIDEO:https://binusianorg-my.sharepoint.com/personal/darrius_salim_binus_ac_id/_layouts/15/guestaccess.aspx?share=EfTm9Ld88ZZMjBhV1dAoLVkBPnxsdbwa9EEGxVatkPO1Ew&nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJPbmVEcml2ZUZvckJ1c2luZXNzIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXciLCJyZWZlcnJhbFZpZXciOiJNeUZpbGVzTGlua0NvcHkifX0&e=4lumiD